In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import URIRef, RDF, RDFS, Graph
import pandas as pd
import numpy as np
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
var=["P36","P38","P37","P122","P31","P463"]
properties=["hasCapital","hasCurrency","officialLanguage","politicalSystem","island nation", "memberOf"]
results=[]
for x in var:
    sparql.setQuery("""
        SELECT ?countryLabel ?article_country  ?article_division ?article_var ?article_alt ?divisionLabel ?varLabel
    WHERE
    {
      ?country wdt:P31 wd:Q3624078 .
      ?country wdt:P30 wd:Q46 .
       OPTIONAL { ?country wdt:P31 wd:Q417175 . 
                ?country wdt:P150 ?division .
                ?division wdt:P30 wd:Q46.
                ?division wdt:"""+x+""" ?var .
                ?article_alt  schema:about ?var .
                ?article_alt  schema:isPartOf <https://en.wikipedia.org/>.
                ?article_division schema:about ?division .
                ?article_division schema:isPartOf <https://en.wikipedia.org/>.
                
               }
      ?article_country schema:about ?country .
      ?article_country schema:isPartOf <https://en.wikipedia.org/>.
      OPTIONAL{?country wdt:"""+x+""" ?var .
      ?article_var schema:about ?var .
      ?article_var schema:isPartOf <https://en.wikipedia.org/>.}
     

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    }
    """)
    sparql.setReturnFormat(JSON)
    results.append(sparql.query().convert())

In [3]:
results_df=[]
for i,e in enumerate(results):
    tmp=pd.json_normalize(results[i]['results']['bindings'])
    for n in range(tmp.shape[0]):
        if(pd.isna(tmp.loc[n]["article_division.value"])==False):
            tmp.loc[n]["article_country.value"]=tmp.loc[n]["article_division.value"]
            tmp.loc[n]["article_var.value"]=tmp.loc[n]["article_alt.value"]
            tmp.loc[n]["countryLabel.value"]=tmp.loc[n]["divisionLabel.value"]
    results_df.append(tmp)
for e,i in enumerate(results_df[3]["varLabel.value"]):
    if (pd.isna(i)!=True):
        results_df[3]["varLabel.value"].loc[e]=results_df[3]["varLabel.value"].loc[e]+" "
results_df[3]=results_df[3].groupby(results_df[3]["countryLabel.value"]).aggregate({"article_country.value":"first","article_var.value":"first","varLabel.value": "sum"})
results_df[3].drop(results_df[3].loc[results_df[3]['varLabel.value']==0].index, inplace=True)
results_df[3]=results_df[3].reset_index()

In [7]:
borders_results=[]
sparql.setQuery("""  SELECT DISTINCT ?countryLabel ?article_country ?article_country2 
    WHERE
    {
      ?border wdt:P31 wd:Q15104814 .
      ?border wdt:P17 ?country .
      ?border wdt:P17 ?country2 .
      ?country wdt:P30 wd:Q46 .
      ?article_country schema:about ?country .
      ?article_country schema:isPartOf <https://en.wikipedia.org/>.
      ?article_country2 schema:about ?country2 .
      ?article_country2 schema:isPartOf <https://en.wikipedia.org/>.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    }
    
""")
sparql.setReturnFormat(JSON)
results=sparql.query().convert()
borders_results_df=pd.json_normalize(results['results']['bindings'])

In [8]:
sparql.setQuery("""  SELECT  ?article_country  ?article_division ?article_var ?article_alt ?instLabel
    WHERE
    {
      ?country wdt:P31 wd:Q3624078 .
      ?country wdt:P30 wd:Q46 .
       OPTIONAL { ?country wdt:P31 wd:Q417175 . 
                ?country wdt:P150 ?division .
                ?division wdt:P30 wd:Q46.
                ?division wdt:P206 ?var .
                ?var wdt:P31 ?inst .
                ?article_alt schema:about ?var .
                ?article_alt schema:isPartOf <https://en.wikipedia.org/>.
                ?article_division schema:about ?division .
                ?article_division schema:isPartOf <https://en.wikipedia.org/>.
                
               }
      ?article_country schema:about ?country .
      ?article_country schema:isPartOf <https://en.wikipedia.org/>.
      OPTIONAL{?country wdt:P206 ?var .
      ?article_var schema:about ?var .
      ?var wdt:P31 ?inst .
      ?article_var schema:isPartOf <https://en.wikipedia.org/>.}
     

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    }
""")
sparql.setReturnFormat(JSON)
water_results=sparql.query().convert()
water_results_df=pd.json_normalize(water_results['results']['bindings'])

In [9]:
for n in range(water_results_df.shape[0]):
    if(pd.isna(water_results_df.loc[n]["article_division.value"])==False):
        water_results_df.loc[n]["article_country.value"]=water_results_df.loc[n]["article_division.value"]
        water_results_df.loc[n]["article_var.value"]=water_results_df.loc[n]["article_alt.value"]

In [22]:
from rdflib import URIRef, RDF, RDFS
g = Graph()
g.parse("progetto.owl", format="xml")
uripro="http://www.semanticweb.org/ress/ontologies/2021/4/country#"
for i,prop in enumerate(properties):
    for e in range(results_df[i].shape[0]):
        to_insert=results_df[i][["article_country.value","article_var.value","varLabel.value","countryLabel.value"]].loc[e]
        if (i==0):
            g.add((URIRef(to_insert[0]),RDF.type,URIRef(uripro+"European_Nation")))
            g.add((URIRef(uripro+"European_Nation"),RDFS.subClassOf,URIRef(uripro+"Nation")))
        if (prop=="politicalSystem"):
            if "monarchy" in to_insert[2]:
                g.add((URIRef(to_insert[1]),RDF.type,URIRef(uripro+"Monarchy")))
                g.add((URIRef(uripro+"Monarchy"),RDFS.subClassOf,URIRef(uripro+"Political_system")))
            elif "republic" in to_insert[2]:
                g.add((URIRef(to_insert[1]),RDF.type,URIRef(uripro+"Republic")))
                g.add((URIRef(uripro+"Republic"),RDFS.subClassOf,URIRef(uripro+"Political_system")))
            else:
                g.add((URIRef(to_insert[1]),RDF.type,URIRef(uripro+"Other")))
                g.add((URIRef(uripro+"Other"),RDFS.subClassOf,URIRef(uripro+"Political_system")))
        if (prop=="island nation"):
            if (to_insert[2]=="island nation") or (borders_results_df.loc[borders_results_df["countryLabel.value"]==to_insert[3]].shape[0]==0):
                g.add((URIRef(to_insert[0]),RDF.type,URIRef(uripro+"Island")))
                g.add((URIRef(uripro+"Island"),RDFS.subClassOf,URIRef(uripro+"Nation")))
            continue
        if (pd.isna(to_insert[2]))==False:
            g.add((URIRef(to_insert[0]),URIRef(uripro+prop),URIRef(to_insert[1])))
g.serialize('country.owl',format='xml')

In [23]:
for e in range(borders_results_df.shape[0]):
    to_insert=borders_results_df[["article_country.value","article_country2.value"]].loc[e]
    if (to_insert[0]!=to_insert[1]):
        g.add((URIRef(to_insert[0]),URIRef(uripro+"confineWith"),URIRef(to_insert[1])))
g.serialize('country.owl',format='xml')

In [24]:
for e in range(water_results_df.shape[0]):
    to_insert=water_results_df[["article_country.value","article_var.value","instLabel.value"]].loc[e]
    if(pd.isna(to_insert["article_var.value"])):
        continue
    if ("sea" in to_insert[2]):
        g.add((URIRef(to_insert[0]),URIRef(uripro+"marineBorderWith"),URIRef(to_insert[1])))
    elif ("ocean" in to_insert[2]):
        g.add((URIRef(to_insert[0]),URIRef(uripro+"marineBorderWith"),URIRef(to_insert[1])))
    elif("river" in to_insert[2]):
        g.add((URIRef(to_insert[0]),URIRef(uripro+"hasRiver"),URIRef(to_insert[1])))
    else:
        g.add((URIRef(to_insert[0]),URIRef(uripro+"hasLake"),URIRef(to_insert[1])))
g.serialize('country.owl',format='xml')